In [0]:
df=dbutils.secrets.get(scope="key-vault-scope",key="app-secrect-key")

In [0]:
dbutils.secrets.listScopes()

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "56c1b601-2d0b-456e-966f-5153657934c6",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="key-vault-scope",key="app-secrect-key"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/fdc798aa-6f15-4e72-bf4a-6c813690fc75/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://tokyo-olympics@adlsgen2tokyoolympics.dfs.core.windows.net/",
  mount_point = "/mnt/tokyo-olympics-data",
  extra_configs = configs)

In [0]:
%fs
ls mnt/tokyo-olympics-data